<a href="https://colab.research.google.com/github/PiyushGithub24/MACHINE_LEARNING_BY_PIYUSH_KUMAR/blob/main/day_18(b)Fetching_Data_From_Web_Scraping(30_05_24).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
import pandas as pd
import requests
from bs4 import BeautifulSoup     #bs4:-beautifulsoup4

In [100]:
requests.get("https://www.ambitionbox.com/list-of-companies?page=1")

<Response [403]>

In [101]:
requests.get("https://www.ambitionbox.com/list-of-companies?page=1").text

'<HTML><HEAD>\n<TITLE>Access Denied</TITLE>\n</HEAD><BODY>\n<H1>Access Denied</H1>\n \nYou don\'t have permission to access "http&#58;&#47;&#47;www&#46;ambitionbox&#46;com&#47;list&#45;of&#45;companies&#63;" on this server.<P>\nReference&#32;&#35;18&#46;4ca7cb17&#46;1717163245&#46;48fd55e9\n<P>https&#58;&#47;&#47;errors&#46;edgesuite&#46;net&#47;18&#46;4ca7cb17&#46;1717163245&#46;48fd55e9</P>\n</BODY>\n</HTML>\n'

In [102]:
url1="https://www.ambitionbox.com/list-of-companies?page=1"

In [103]:
def create_df(url):
  headers=headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
  webpage=requests.get(url,headers=headers).text
  soup=BeautifulSoup(webpage,'lxml')
  company=soup.find_all('div',class_='companyCardWrapper')

  names=[]
  reviews=[]
  rating=[]
  industry=[]
  employees=[]
  ctype=[]
  old=[]
  hq=[]

  for i in company:
    names.append(i.find('h2').text.strip())
    rating.append(i.find('span',class_="companyCardWrapper__companyRatingValue").text)
    reviews.append(i.find('span',class_="companyCardWrapper__ActionCount").text)
    info=(i.find('span', class_="companyCardWrapper__interLinking")).text.strip()
    info=info.split('|')
    if len(info)==5:
      industry.append(info[0])
      employees.append(info[1])
      ctype.append(info[2])
      old.append(info[3])
      hq.append(info[4])
    else:
      for i in info:
        if 'Employees' in i:
          employees.append(i)
          break
      else:
          employees.append(None)

      for i in info:
        if 'old' in i:
          old.append(i)
          break
      else:
          old.append(None)

      for i in info:
        if "more" in i:
          hq.append(i)
          break
      else:
          hq.append(None)

      industry.append(info[0])

      c=['Public','Startup','Central','Private ','Partnership','Cooperative']
      found=False
      for element in c:
        for sentence in info:
            if element in sentence:
              ctype.append(sentence)
              found=True
              break
        if found:
          break
      else:
        ctype.append(None)
  comp_dict={'Company_names':names,"rating":rating,"Reviews":reviews,"Industry":industry,"Employess":employees,"Company_type":ctype,"Company_Old_Years":old,"Headquarter":hq}
  comp_df=pd.DataFrame(comp_dict)
  return comp_df

In [104]:
company_dataframe=pd.DataFrame()
for i in range(1,50):
  url="https://www.ambitionbox.com/list-of-companies?page={}".format(i)
  company_dataframe=pd.concat([company_dataframe,create_df(url)],ignore_index=True)


In [105]:
company_dataframe.shape

(980, 8)

In [106]:
company_dataframe.head()

,Company_names,rating,Reviews,Industry,Employess,Company_type,Company_Old_Years,Headquarter
0,TCS,3.8,75k,IT Services & Consulting,1 Lakh+ Employees,Public,56 years old,Mumbai +346 more
1,Accenture,4.0,46.7k,IT Services & Consulting,1 Lakh+ Employees,Public,35 years old,Dublin +180 more
2,Cognizant,3.9,42k,IT Services & Consulting,1 Lakh+ Employees,Forbes Global 2000,30 years old,Teaneck. New Jersey. +169 more
3,Wipro,3.8,40.3k,IT Services & Consulting,1 Lakh+ Employees,Public,79 years old,Bangalore/Bengaluru +274 more
4,Capgemini,3.8,34.6k,IT Services & Consulting,1 Lakh+ Employees,Public,57 years old,Paris +139 more


In [107]:
company_dataframe.tail()

,Company_names,rating,Reviews,Industry,Employess,Company_type,Company_Old_Years,Headquarter
975,MAHLE ANAND Filter Systems,4.2,707,Auto Components,1k-5k Employees,None,19 years old,Stuttgart +25 more
976,Teva Pharmaceuticals,4.2,722,Pharma,1k-5k Employees,Forbes Global 2000,123 years old,Dublin +33 more
977,INDO-MIM,4.3,686,Industrial Machinery,1k-5k Employees,None,28 years old,Bangalore/Bengaluru +19 more
978,Capital Numbers Infotech,4.7,741,IT Services & Consulting,501-1k Employees,None,12 years old,Kolkata +16 more
979,Birla Corporation,4.2,716,Building Material,5k-10k Employees,Public,105 years old,Kolkata +111 more


In [108]:
company_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 980 entries, 0 to 979
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Company_names      980 non-null    object
 1   rating             980 non-null    object
 2   Reviews            980 non-null    object
 3   Industry           980 non-null    object
 4   Employess          974 non-null    object
 5   Company_type       591 non-null    object
 6   Company_Old_Years  952 non-null    object
 7   Headquarter        980 non-null    object
dtypes: object(8)
memory usage: 61.4+ KB


#Now I successfully fetch all required data of companies from many pages!!!

In [109]:
#Let's create csv file of the dataframe:--
company_dataframe.to_csv('Companies_Related_Data.csv')